In [1]:
import os
import time
from dotenv import load_dotenv
import requests
import pandas as pd
import numpy as np
from requests.exceptions import RequestException

# Load environment variables from the specific .env file
load_dotenv()

# Get API key from environment variable
api_key = os.getenv('CENSUS_API_KEY')

# Define human-readable column names
column_names = {
    'NAME': 'Metro Area',
    'B08303_001E': 'Total',
    'B08303_002E': 'Less than 5 minutes',
    'B08303_003E': '5 to 9 minutes',
    'B08303_004E': '10 to 14 minutes',
    'B08303_005E': '15 to 19 minutes',
    'B08303_006E': '20 to 24 minutes',
    'B08303_007E': '25 to 29 minutes',
    'B08303_008E': '30 to 34 minutes',
    'B08303_009E': '35 to 39 minutes',
    'B08303_010E': '40 to 44 minutes',
    'B08303_011E': '45 to 59 minutes',
    'B08303_012E': '60 to 89 minutes',
    'B08303_013E': '90 or more minutes'
}

def fetch_and_save_data(year, max_retries=3, delay=5):
    # Census API endpoint
    api_url = f"https://api.census.gov/data/{year}/acs/acs1"

    # Parameters for the API request
    params = {
        "get": ",".join(column_names.keys()),
        "for": "metropolitan statistical area/micropolitan statistical area:*",
        "key": api_key
    }

    for attempt in range(max_retries):
        try:
            # Make the API request
            response = requests.get(api_url, params=params, timeout=30)
            response.raise_for_status()  # Raise an exception for bad status codes

            # Process the data
            data = response.json()
            df = pd.DataFrame(data[1:], columns=data[0])
            
            # Rename columns to human-readable names
            df = df.rename(columns=column_names)
            
            # Convert numeric columns to integers, replacing non-numeric values with NaN
            numeric_columns = df.columns[1:-1]  # All columns except 'Metro Area' and the last one
            for col in numeric_columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # Replace NaN with 0 or handle as needed
            df[numeric_columns] = df[numeric_columns].fillna(0).astype(int)
            
            # Remove the metropolitan statistical area/micropolitan statistical area code column
            df = df.drop(columns=['metropolitan statistical area/micropolitan statistical area'])
            
            # Add a Year column
            df['Year'] = year
            
            # Save to CSV
            output_file = f'Commute_Time_By_Metro_Area_{year}.csv'
            df.to_csv(output_file, index=False)
            print(f"Data for {year} saved to {os.path.abspath(output_file)}")
            return df

        except RequestException as e:
            print(f"Attempt {attempt + 1} failed for year {year}. Error: {e}")
            if attempt < max_retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print(f"Failed to fetch data for {year} after {max_retries} attempts.")
                return None

# Years to fetch data for
years = range(2010, 2025)  # 2010 to 2024

all_data = []

for year in years:
    print(f"Fetching data for {year}...")
    year_data = fetch_and_save_data(year)
    if year_data is not None:
        all_data.append(year_data)
    else:
        print(f"Skipping {year} due to persistent errors.")
    print()  # Add a blank line for readability
    time.sleep(2)  # Add a 2-second delay between years to avoid overwhelming the API

# Combine all data into a single DataFrame
if all_data:
    global_df = pd.concat(all_data, ignore_index=True)
    
    # Save the global CSV
    global_csv = 'Commute_Time_By_Metro_Area_All_Years.csv'
    global_df.to_csv(global_csv, index=False)
    print(f"Global data for all years saved to {os.path.abspath(global_csv)}")
else:
    print("No data was successfully retrieved for any year.")

print("Data fetching complete.")

Fetching data for 2010...
Data for 2010 saved to /Users/trevor/coding/repos/ai-project-2/Commute_Time_By_Metro_Area_2010.csv

Fetching data for 2011...
Data for 2011 saved to /Users/trevor/coding/repos/ai-project-2/Commute_Time_By_Metro_Area_2011.csv

Fetching data for 2012...
Data for 2012 saved to /Users/trevor/coding/repos/ai-project-2/Commute_Time_By_Metro_Area_2012.csv

Fetching data for 2013...
Data for 2013 saved to /Users/trevor/coding/repos/ai-project-2/Commute_Time_By_Metro_Area_2013.csv

Fetching data for 2014...
Data for 2014 saved to /Users/trevor/coding/repos/ai-project-2/Commute_Time_By_Metro_Area_2014.csv

Fetching data for 2015...
Data for 2015 saved to /Users/trevor/coding/repos/ai-project-2/Commute_Time_By_Metro_Area_2015.csv

Fetching data for 2016...
Data for 2016 saved to /Users/trevor/coding/repos/ai-project-2/Commute_Time_By_Metro_Area_2016.csv

Fetching data for 2017...
Data for 2017 saved to /Users/trevor/coding/repos/ai-project-2/Commute_Time_By_Metro_Area_20

In [2]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('Commute_Time_By_Metro_Area_All_Years.csv')

# Group by Metro Area and calculate the average Total
metro_averages = df.groupby('Metro Area')['Total'].mean().reset_index()

# Sort by Total in descending order and select top 25
top_25_metros = metro_averages.sort_values('Total', ascending=False).head(25)

# Get the list of top 25 metro areas
top_25_metro_list = top_25_metros['Metro Area'].tolist()

# Filter the original dataframe to include only the top 25 metro areas
top_25_data = df[df['Metro Area'].isin(top_25_metro_list)]

# Sort the data by Metro Area and any other relevant columns (e.g., year if present)
top_25_data = top_25_data.sort_values(['Metro Area'])

# Save the results to a new CSV file
top_25_data.to_csv('Commute_Time_By_Top_25_Metro_Area_All_Years.csv', index=False)

print(f"Data for top 25 metro areas has been saved to 'Commute_Time_By_Top_25_Metro_Area_All_Years.csv'")

Data for top 25 metro areas has been saved to 'Commute_Time_By_Top_25_Metro_Area_All_Years.csv'


In [3]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('Commute_Time_By_Top_25_Metro_Area_All_Years.csv')

# List of columns to convert to percentages
time_columns = [
    "Less than 5 minutes", "5 to 9 minutes", "10 to 14 minutes",
    "15 to 19 minutes", "20 to 24 minutes", "25 to 29 minutes",
    "30 to 34 minutes", "35 to 39 minutes", "40 to 44 minutes",
    "45 to 59 minutes", "60 to 89 minutes", "90 or more minutes"
]

# Calculate percentages
for col in time_columns:
    df[col] = (df[col] / df['Total'] * 100).round(5)

# Save the results to a new CSV file
df.to_csv('Commute_Time_By_Top_25_Metro_Area_All_Years_Percentage.csv', index=False)

print("Percentage data has been saved to 'Commute_Time_By_Top_25_Metro_Area_All_Years_Percentage.csv'")

Percentage data has been saved to 'Commute_Time_By_Top_25_Metro_Area_All_Years_Percentage.csv'


In [4]:
# What this Code has Done is create a csv/Dataframe that is called "Commute_Time_By_Top_25_Metro_Areas_All_Years.csv"
# For the top 25 by size it contains for each year from 2010 through 2024 (2023 actually, because '24 data not available) it has the
# commute times brokwn down into different 5/10 minute categories
#  To make this usable, we're going to then ask it to convert the raw numbers into percentages so we can see how
# much of a particular population is doing very long commutes, how many slower commutes


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Read the CSV file
df = pd.read_csv('Commute_Time_By_Top_25_Metro_Area_All_Years_Percentage.csv')

# Ensure 'Year' column exists
if 'Year' not in df.columns:
    print("Error: 'Year' column not found in the CSV file.")
    exit()

# List of time categories
time_categories = [
    "Less than 5 minutes", "5 to 9 minutes", "10 to 14 minutes",
    "15 to 19 minutes", "20 to 24 minutes", "25 to 29 minutes",
    "30 to 34 minutes", "35 to 39 minutes", "40 to 44 minutes",
    "45 to 59 minutes", "60 to 89 minutes", "90 or more minutes"
]

# Create a directory to store the graphs
if not os.path.exists('metro_graphs'):
    os.makedirs('metro_graphs')

# Function to create a graph for a single metro area
def create_metro_graph(metro_data, metro_name):
    plt.figure(figsize=(15, 10))
    for category in time_categories:
        plt.plot(metro_data['Year'], metro_data[category], label=category)
    
    plt.title(f'Commute Time Percentages for {metro_name} (2005-2018)')
    plt.xlabel('Year')
    plt.ylabel('Percentage')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.savefig(f'metro_graphs/{metro_name.replace("/", "_")}_commute_times.png')
    plt.close()

# Group by Metro Area and create graphs
for metro_area, group in df.groupby('Metro Area'):
    create_metro_graph(group, metro_area)

print("Graphs have been saved in the 'metro_graphs' directory.")

Graphs have been saved in the 'metro_graphs' directory.
